[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adan-rs/amd/blob/main/notebooks/08_Tablas_cruzadas.ipynb)

# Tablas cruzadas
Una forma básica de realizar una tabla cruzada es mediante el método crosstab de Pandas, especificando las variables en las filas y columnas respectivamente. Por ejemplo, para las variables "A" y "B", utilizaremos:  
`pd.crosstab(df['A'], df['B'])`  
El argumento *margins* se puede agregar para mostrar la suma por fila o columna. Cada celda dentro de la tabla mostrará la frecuencia observada en esa intersección

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#df = pd.read_excel('../data/enigh2020.xlsx')
df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/enigh2020.xlsx')

In [ ]:
# Crea una tabla cruzada para relacionar "sexo_jefe" con "est_socio"
tabla = pd.crosstab(df['sexo_jefe'], df['est_socio'], margins=False)
tabla

En muchas ocasiones se requiere representar esta tabla cruzada mediante proporciones. Para ello se puede utilizar el argumento *normalize* con los siguientes valores:

    'all': para dividir cada cantidad entre el total.
    'index': para dividir las celdas en cada fila entre el total de la fila.
    'columns': para dividir las celdas en cada columna entre el total de la columna.

In [ ]:
# Muestra una tabla con las cantidades *normalizadas* entre el total de la columna
tabla_normalizada = pd.crosstab(df['sexo_jefe'], df['est_socio'], normalize='index')
tabla_normalizada

In [ ]:
# Ejemplo de reordenar columnas
orden_columnas = ['bajo', 'medio_bajo', 'medio_alto', 'alto']
tabla_normalizada = tabla_normalizada[orden_columnas]
tabla_normalizada

Para visualizar la relación entre ambas variables se puede utilizar un "mapa de calor". Es recomendable utilizar la librería *seaborn* como en el siguiente ejemplo:  
`sns.heatmap(tabla, annot=True, cmap='YlGnBu', fmt='d')`  
*annot=True* sirve para indicar la cantidad en cada celda, *fmt=d* se utiliza para que se despliegue como números enteros, y *cmap* se utiliza para seleccionar una paleta de colores (ver https://matplotlib.org/stable/gallery/color/colormap_reference.html)

In [ ]:
plt.figure(figsize=(6, 2)) 
sns.heatmap(tabla_normalizada,annot=True, cmap='Greens');

# Matriz de correlaciones
Un medida útil para evaluar el grado de relación lineal entre dos variables cuantitativas es el coeficiente de correlación de Pearson. Este coeficiente puede tomar valores entre -1 y 1, donde valores positivos indican una relación directa, valores negativos indican una relación inversa, y cuanto más cercano sea a 0 más débil es esta relación. 

Para calcular un coeficiente de correlación de Pearson entre dos columnas se puede utilizar el método *corr()* de Pandas. Por ejemplo:
`correlacion = df['X'].corr(df['Y'])`  

Para calcular el coeficiente de correlación de Spearman se puede agregar como argumento:  
`correlacion = df['X'].corr(df['Y'], method='spearman')`  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/enigh2020.xlsx')

In [ ]:
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
df = remove_outliers_iso_forest(df, ['ing_cor', 'gasto_mon'])

In [ ]:
df.plot(kind='scatter', x='ing_cor', y='gasto_mon');

El coeficiente de correlación es una medida del grado o fuerza de la relación lineal entre dos variables cuantitativas. Un valor positivo del coeficiente de correlación indica una relación directa mientras que un valor negativo indica una relación inversa.

In [ ]:
# Calcula la correlación entre "ing_cor" y "gasto_mon"
correlacion = df['ing_cor'].corr(df['gasto_mon'])
correlacion.round(4)

Si se tiene un DataFrame df de variables cuantitativas se puede obtener la matriz de correlaciones utilizando df.corr()


In [ ]:
var_cont = ["ing_cor", "gasto_mon", "edad_jefe", "tot_integ"]
matriz_corr = df[var_cont].corr()
matriz_corr

In [ ]:
plt.figure(figsize=(6, 4)) 
sns.heatmap(matriz_corr,annot=True, cmap='Greens');
plt.title("Mapa de calor", fontsize=16) 